In [71]:
pip install -U langchain

Note: you may need to restart the kernel to use updated packages.


### 교차반응목록

In [1]:
import pandas as pd
# df = pd.read_csv('data/교차반응식품.csv')
# df

### 요리명, 요리제목, 일련번호

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='data/레시피이름_url번호.csv', encoding='utf-8')
data = loader.load()

print(len(data))

182808


In [3]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=0,
    encoding_name='cl100k_base'
)
documents = text_splitter.split_documents(data)
len(documents)

182808

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings


In [5]:


embeddings_model = HuggingFaceEmbeddings(
    # model_name='jhgan/ko-sroberta-multitask',
        # model_name='jhgan/ko-sbert-nli',
        model_name='BM-K/KoSimCSE-RoBERTa-multitask',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

C:\ProgramData\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\ProgramData\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


No sentence-transformers model found with name BM-K/KoSimCSE-RoBERTa-multitask. Creating a new one with mean pooling.


In [6]:
# load db
vectorstore = FAISS.load_local('./db/faiss_model_KoSimCSE', embeddings_model,
                     allow_dangerous_deserialization=True)


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders.csv_loader import CSVLoader

```

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders.csv_loader import CSVLoader

```

#### 함수: 알레르기가 없는 레시피 번호, 교차반응 메세지, 가공식품에 알레르기가 포함된 메세지 모듈들

In [9]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.callbacks.base import BaseCallbackHandler
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


import pandas as pd
recipe_df = pd.read_csv('data/레시피_일련번호.csv')
ingredient_df = pd.read_csv('data/레시피_재료.csv')
raw_material_df = pd.read_csv('data/원재료.csv')
processed_food_df = pd.read_csv('data/가공식품.csv')
ing_name_df = pd.read_csv('data/재료명2_수정.csv')
# 이미지 url
img_df = pd.read_csv('data/recipe_content_final.csv')

In [10]:
img_df.head(2)

,일련번호,이미지
0,1,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,1,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [11]:
recipe_df.head(2)

,URL_Num,Recipe_Num,Recipe_Name,Recipe_Title
0,128671,1,어묵김말이,어묵김말이
1,128892,2,두부새우전,두부에 꼬리가 달렸어요!!


In [12]:
ingredient_df.head(2) 

,Recipe_Ingredients_ID,Recipe_Num,Ingredients_Code,Ingredient_EA
0,0,1,I04330,2개
1,1,1,I00608,3장


In [13]:
raw_material_df.head(2) 

,Ingredients_Allergy_ID,Ingredients_Code,Allergy_Code,Product_Check
0,12,I00002,NaN,0
1,14,I00004,NaN,0


In [14]:
processed_food_df.head(2) 

,Ingredients_Allergy_ID,Ingredients_Code,Allergy_Code,Product_Check
0,0,I00000,A04,1
1,1,I00000,A03,1


In [15]:
ing_name_df.head(2)

,Ingredients_Code,Ingredient_Name,Product_Check,Recipe_Ingredients_ID,Recipe_Num,Ingredient_EA
0,I00000,핫도그,1,78582,7518,5개
1,I00000,핫도그,1,382794,40022,2개


In [16]:
recipe_df = pd.read_csv('data/레시피_일련번호.csv')
ingredient_df = pd.read_csv('data/레시피_재료.csv')
raw_material_df = pd.read_csv('data/원재료.csv')
processed_food_df 

,Ingredients_Allergy_ID,Ingredients_Code,Allergy_Code,Product_Check
0,0,I00000,A04,1
1,1,I00000,A03,1
2,2,I00000,A01,1
3,3,I00000,A17,1
4,4,I00000,A14,1
...,...,...,...,...
7770,11113,I06610,A09,1
7771,11114,I06610,A13,1
7772,11115,I06611,A14,1
7773,11116,I06611,A05,1


#### 함수: 알레르기가 없는 레시피 번호,url일련번호, 교차반응 메세지, 가공식품에 알레르기가 포함된 메세지
- CrossAndWarn(user_food, context, allergy)
- return: 레시피번호, url일련번호(str), 원재료와 가공식품의 교차반응 메세지(딕셔너리),  가공식품에 알레르기가 포함된 메세지(딕셔너리)

In [17]:
    
def CrossAndWarn(user_food,context,allergys=None):
    import re
   

    c_temp = [doc for doc in context  if re.search(fr'{user_food}', doc.page_content)]
    # 벡터스토어에 요리명이 아예 없는경우 (ex. 비건탕수육)
    if len(c_temp) == 0:
        return -1,'-1',{},{}

    
     # 셔플 # 사용자에게 더욱 다양하게 주기위해서 셔플
    import random
    random.shuffle(c_temp)

    
    # 일반쿼리
    recipe_numbers_li = [re.search(r'레시피번호:\s*(\d+)', doc.page_content).group(1) for doc in c_temp]
    recipe_numbers_li =[int(i) for i in recipe_numbers_li ] 

    
    # 셔플
    import random
    random.shuffle(recipe_numbers_li)

    
    cross = pd.read_csv('data/알레르기_교차반응_매핑_수정본.csv')
    cross.drop(columns= ['Unnamed: 0','알레르기식품명','교차반응식품명'],inplace=True)
    c = cross.set_index(['알레르기코드']).to_dict()

    # temp는 이중리스트구조 [['A09', 'C29', 'C15'], ['A05', 'A06', 'A07', 'C27', 'C10', 'C17', 'C03']]를 다시 일차원 리스트로
    temp =[c['교차반응코드'][allergy].split(',') for allergy in allergys]

    # c_li는 교차반응코드를 중복없이 받을꺼임
    c_li= []
    
    for li in temp:
        for i in li:
            c_li.append(i)

    c_li = list(set(c_li))
            


    
    count=1
    # 레시피번호 하나씩 확인하는 절차
    for num in recipe_numbers_li:
        print(f'{count}번째')
        count+=1
        print(f'레시피 번호 : {num}')

        # cnt=0이면 원재료에 알레르기성분 있다 cnt=1이면 정상 cnt=-1이면 못찾음
        cnt=-1
        # 재료df에서 레시피번호에 해당하는 요리
        temp_df= ingredient_df[ingredient_df['Recipe_Num'].isin([num])]
        
        # 재료들을 리스트에 넣어둠
        ingred_li = ingredient_df[ingredient_df['Recipe_Num'].isin([num])]['Ingredients_Code']
        temp_li = []
       
        
        temp_raw_dict = {} # 원재료 교차반응 담을꺼임 ex){ 재료1:[코드1,코드2...], ... }
        temp_pre_dict = {} # 가공식품 교차반응 담을꺼임 ex){ 재료2:[코드1,코드2...], ... }
        warning = {}
        
        # 알레르기 가공품에 해당하는 목록들을 모음
        temp_preprocess = []

        # 알러지가 없으면 넘어가자~~
        if allergys == None:
            break
        
        # 각각의 재료
        print('원재료 확인 시작')
        for ingredient in list(ingred_li):
            print(ingredient)
            # 교차반응 코드 넣음 (딕셔너리에 
            temp_li = []
    
            # 원재료 비교
            # 재료에 해당하는 코드(알레르기와 비교할꺼임) 
            for code in list(raw_material_df[raw_material_df['Ingredients_Code'] == ingredient]['Allergy_Code']):
                # 알레르기코드와 원재료 알레르기코드와 같으면 끝. 
                if code in allergys :
                    cnt=0
                    break
                if code in c_li:
                    temp_li.append(code)
            
            temp_raw_dict[ingredient]= temp_li
            
            if cnt == 0:
                print(f'레시피 번호 {num}에 재료 {ingredient}에 알레르기 성분 존재.')
                break
        if cnt == 0 :
            print('다음 번호 확인')
            continue
    
        # 재료를 가공식품 테이븝에서 비교
        print('가공식품 확인 시작')
        cnt = 1
        for ingredient in list(ingred_li):    
            print(ingredient)
            
             # 알레르기 가공식품 경고
            temp_warn = []
            temp_li = []
    
            
            for code in list(processed_food_df[processed_food_df['Ingredients_Code'] == ingredient]['Allergy_Code']):
               
                if code in allergys :
                    temp_warn.append(code)
    
                 
                if code in c_li:
                    temp_li.append(code)
           
            temp_pre_dict[ingredient] = temp_li
            warning[ingredient] = temp_warn
            
    
        
        if cnt == 1:
            print('문제 없음')
            print('끝')
            break

    if cnt==1:
        combined = {}
        for key in temp_raw_dict.keys():
            combined[key] = temp_raw_dict[key] + temp_pre_dict[key]

            url_num = recipe_df[recipe_df['Recipe_Num'] ==num]['URL_Num']
            u = url_num.values[0]
            
            # 레시피 번호, 교차반응 메세지, 가공식품에 알레르기가 포함된 메세지
        return num,u,combined,warning
            
    else:
        return -1,'-1',{},{}
        
  
            

### 위의 함수에서 받은 경고가 code(ex.A04)로 구현되있기때문에 이름으로 바꿔준다
- 입력: 원재료와 가공식품의 교차반응 메세지(딕셔너리), 가공식품에 알레르기가 포함된 메세지(딕셔너리)
- return : 위의 입력을 한글로 대체하고 경고 문구를 return 해준다

In [18]:
def replaceCode(oto_dict,cross_message,warning_message,allergys=None):
  
    cross_comment_li = {}
    warn_comment_li= {}
    
    #알러지 코드도 한글로
    c= []
    for allergy in allergys:
        c.append(oto_dict[allergy])
    
    if allergys == None:
        return c,cross_comment_li, warn_comment_li
    
    for ikey,ival in cross_message.items():
        if ival:
            # 재료코드를 재료이름으로 바꾸기
            code_Name= ing_name_df[(ing_name_df['Recipe_Num'] == recipe_num) & (ing_name_df['Ingredients_Code'] == ikey)]['Ingredient_Name']
            code_Name= code_Name.values[0]
            
            # 유발 식품 넣어두기
            food_names= []
            for val in ival:
                food_names.append(oto_dict[val])
            cross_comment_li[code_Name] = food_names
    
    for ikey,ival in warning_message.items():
        if ival:
            # 재료코드를 이름으로 바꾸기
            code_Name= ing_name_df[(ing_name_df['Recipe_Num'] == recipe_num) & (ing_name_df['Ingredients_Code'] == ikey)]['Ingredient_Name']
            code_Name= code_Name.values[0]
            
            # 유발 식품 넣어두기
            food_names= []
            for val in ival:
                food_names.append(oto_dict[val])
            warn_comment_li[code_Name] = food_names
    
    return c, cross_comment_li, warn_comment_li
        

### 필요X 그냥 재료 검수용 함수

In [19]:
def recipe_dff(number):
    print(recipe_df[recipe_df['Recipe_Num'] ==number])
def ingredeint_dff(number):
    print(ingredient_df[ingredient_df['Recipe_Num'] ==number])
def raw_material_dff(number):
    print(raw_material_df[raw_material_df['Ingredients_Code'] ==number])
def processed_food_dff(number):
    print(processed_food_df[processed_food_df['Ingredients_Code'] ==number])

```

### 일대일 알레르기,코드 매핑

In [20]:
oto_df = pd.read_csv('data/식품_코드_일대일매칭.csv')
oto_df.rename(columns = {'Unnamed: 0':'코드'},inplace=True)
oto_dict = oto_df.set_index('코드').to_dict()
oto_dict = oto_dict['식품명']
oto_dict

{'A01': '난류',
 'A02': '소고기',
 'A03': '돼지고기',
 'A04': '닭고기',
 'A05': '새우',
 'A06': '게',
 'A07': '오징어',
 'A08': '고등어',
 'A09': '조개류',
 'A10': '우유',
 'A11': '땅콩',
 'A12': '호두',
 'A13': '잣',
 'A14': '대두',
 'A15': '복숭아',
 'A16': '토마토',
 'A17': '밀',
 'A18': '메밀',
 'A19': '아황산류',
 'C16': '우유',
 'C28': '참깨',
 'C34': '닭고기',
 'C37': '돼지고기',
 'C14': '소고기',
 'C30': '바닷가재',
 'C32': '꽃게',
 'C17': '굴',
 'C27': '새우',
 'C03': '오징어',
 'C15': '전복',
 'C24': '송이버섯',
 'C29': '연어',
 'C10': '게',
 'C25': '염소젖',
 'C06': '말젖',
 'C05': '대두',
 'C26': '완두콩',
 'C35': '렌즈콩',
 'C12': '브라질너트',
 'C01': '캐슈넛',
 'C11': '헤이즐넛',
 'C38': '호두',
 'C33': '아몬드',
 'C09': '녹두',
 'C13': '강낭콩',
 'C21': '땅콩',
 'C04': '사과',
 'C19': '자두',
 'C22': '체리',
 'C40': '배',
 'C07': '호밀풀',
 'C31': '셀러리',
 'C20': '쑥',
 'C08': '감자',
 'C36': '호밀',
 'C39': '보리',
 'C23': '밀',
 'C02': '건포도',
 'C18': '백포도주'}

```

### 웹베이스 로더 추가 멀티체인

In [21]:
class User:
    allergies=''
    user_food=''
    
user1 = User()

In [38]:
import math
import time

start = time.time()
math.factorial(100000)


import ast





# Retrieval
# 요리명 일련번호 관련 벡터에서 찾기
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 500, 'fetch_k': 800,'lambda_mult': 0.9}
)



ans = """ https://www.10000recipe.com/recipe/’’’insert here’’’"""
ans2 =  """{'name':’’’insert here’’’,'ingredients':’’’insert here’’’,'recipe':[’’’insert here’’’]}"""
ans3 = """{'allergy':'''insert here''','food':'''insert here'''}"""

# Prompt
template3 = '''

Question : {question}


사용자가 알레르기와 요리명 정보를 전달할거야. 맞춤법은 고쳐줘.
만약 사용자의 알레르기 정보가 주어지면 알레르기를 리스트에서 참고해줘.
참고로 계란,달걀,반숙은 난류야.
['소고기','난류','돼지고기','닭고기','새우','게','오징어','고등어','조개류','우유','땅콩',
 '호두','잣','대두','복숭아','토마토','밀','메밀','아황산류']
꼭 알레르기와 질문은 분리해야해.
food에는 알레르기 성분들을 제외한 핵심적인 요리이름만 넣어줘!.
사용자가 알레르기 정보를 주지 않는다면 {prev_allergy}를 알레르기 정보로 사용해.
다른거 추천해달라하면 {prev_food}를 요리명에 넣어줘
쓸데없는 말 넣지말고 다음 형식을 따라서 대답해줘:
{ans3}
'''


template2 = ''' {site_document} 참고해서 요리제목, 재료,수량, 레시피를 한국어로 알려줘.

{ans2}
'''

template4 = '''
        Extract and normalize the following list of ingredients according to these conditions:
        1. Choose one when ingredients are connected with '또는', '~나', 'or' 
        2. Use the same name for similar ingredients.
        3. Ensure the core ingredient remains.
        4. Make sure the number of ingredients in the output is the same as in the input list of ingredients
        5. Ensure that the input and output ingredients are in a 1:1 correspondence
        
        Provide a Dictionary in which original ingredient names are keys and normalized ingredient names are values.
        Output must be korean
        Just response Dictionary
        
        {ingredients}


'''
template__0 = '''
사용자에게 {allergy}가 없는 {user_food} 레시피 알려줘
'''


# prompt = ChatPromptTemplate.from_template(template)
prompt2 = ChatPromptTemplate.from_template(template2)
prompt3 = ChatPromptTemplate.from_template(template3)
prompt4 = ChatPromptTemplate.from_template(template4)
prompt__0 = ChatPromptTemplate.from_template(template__0)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0.6,
    max_tokens=850,
)
llm4 = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.2,
    max_tokens=850,
)




# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])



model_parser = llm | StrOutputParser()
model_parser4 = llm4 | StrOutputParser()


#####################################################################################################
query = '밀 알레르기가 있어 라면 알려줘'
#####################################################################################################



# Chain
chain3 = (prompt3 
          |  model_parser4)
al_food_json = chain3.invoke({'question': query,'prev_allergy':user1.allergies,'prev_food':user1.user_food,'ans3':ans3})
al_food_dict =  ast.literal_eval(al_food_json)

# '조개류, 고등어' 인경우 공백이 문제가되므로 공백도 제거해준다


if type(al_food_dict['allergy']) == list:
    # 이 코드를 str일때도 해주면 '돼','지','고','기' 같은 형태가 나와서 분리해줌
    allergys =[ a.strip(' ') for a in ','.join(al_food_dict['allergy']).split(',')]
elif type(al_food_dict['allergy']) == str:
    allergys =[ a.strip(' ') for a in al_food_dict['allergy'].split(',')]

print(f'allergys : {type(allergys)}')

user1.allergies = allergys
al_code = []

for allergy in allergys:
    for a,b in oto_dict.items():
        if b== allergy:
            al_code.append(a)
            break
print(f'allergys:{allergys}')
print(f'al_code:{al_code}')
user_food = al_food_dict['food']
user1.user_food = user_food
print(f'사용자 입력 음식: {user_food}')

# unique_docs = retriever_from_llm.get_relevant_documents(query=user_food)
# context =  format_docs(unique_docs)
print('벡터스토어에서 가저오는중...')
context = retriever.get_relevant_documents(query=user_food)
print('검색 완료...!')
# context2 =  format_docs(data2)


recipe_num, url_num,cross_message, warning_message = CrossAndWarn(user_food,context,al_code)
# 해당하는 음식이 없을 경우
if recipe_num == -1:
    print('죄송합니다. GPT 3.5를 사용해서 레시피 추천해드릴께요.')
    chain__0 = (prompt__0 
          |  model_parser)
    ex_food = chain__0.invoke({'allergy': allergys,'user_food': user_food})
    print(ex_food)

else:
    site = 'https://www.10000recipe.com/recipe/'+f'{url_num}'
    
    print('Loading')
    import bs4
    from langchain_community.document_loaders import WebBaseLoader
    loader = WebBaseLoader(
        web_paths=(site,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("view2_summary_in", "ready_ingre3","view_step"),
                id=('recipeIntro','divConfirmedMaterialArea','obx_recipe_step_start',)
            )
        ),
    )
    site_document = loader.load()
    
    # )
    
    chain2 =( prompt2 
             | model_parser
    )
    
    response_temp = chain2.invoke({ 'site_document':site_document,'ans2':ans2})
    response = ast.literal_eval(response_temp)
    
    
    print(f"요리제목: {response['name']}\n")
    print(f"재료: {response['ingredients']}\n")
    print(f"레시피:\n")
    for i,re in enumerate(response['recipe']):
        print(f'  {i+1}. {re}')
    
    # # 교차반응 얘기해주기
    c,cross_comment_li,warn_comment_li = replaceCode(oto_dict,cross_message,warning_message,al_code)
    if (cross_comment_li or warn_comment_li):
        print('\n※주의사항\n')
        if cross_comment_li:
            print(f'사용자의 알레르기식품({c})은')
            for c_key,c_val in cross_comment_li.items():
                print(f'{c_key}({c_val}함유) 와 교차반응을 일으킬 수 있습니다!.')
        if warn_comment_li:
            for ke,va in warn_comment_li.items():
                print(f'\n가공식품 {ke}에 {va}가 함유 되있습니다!.')

    print(f'\n출처 사이트: {site}')


    print(f'allergys : {allergys}')
    print()
    print(f'al_food_json : {al_food_json}')
    print()
    print(f'al_food_dict : {al_food_dict}')
    print()
    print(f'user_food : {user_food}')
    print()
    print(f'site : {site}')
    print()
    print(f'response : {response}')
    
       

end = time.time()

print(f"{end - start:.5f} sec")



allergys : <class 'list'>
allergys:['밀']
al_code:['A17']
사용자 입력 음식: 라면
벡터스토어에서 가저오는중...
검색 완료...!
1번째
레시피 번호 : 93211
원재료 확인 시작
I01801
I00319
I04347
I00613
I00003
I00797
가공식품 확인 시작
I01801
I00319
I04347
I00613
I00003
I00797
문제 없음
끝
Loading
요리제목: 냉라면

재료: 라면 1봉지, 양파 1/4개, 얼음 적당량, 설탕 1큰술, 식초 1큰술, 참기름 1큰술

레시피:

  1. 양파는 얇게 채 썰어 찬물에 담가 매운기를 뺀다.
  2. 끓는 물에 면과 후레이크를 삶는다.
  3. 후레이크는 기호에 따라 빼셔도 돼요.
  4. 면과 후레이크가 다 삶아지면 찬물에 헹구고 체에 받쳐 물기를 뺀다.
  5. 뜨거운 물 50ml에 라면 스프를 녹인다.
  6. 스프를 푼 물에 식초, 설탕, 참기름을 1큰술씩 넣는다.
  7. 그릇에 물기를 뺀 면과 후레이크를 넣는다.
  8. 면에 준비한 스프 양념장을 붓는다.
  9. 찬물과 200ml와 얼음을 붓는다.
  10. 채썬 양파를 올리면 냉라면이 완성된다.

※주의사항


가공식품 라면에 ['밀']가 함유 되있습니다!.

출처 사이트: https://www.10000recipe.com/recipe/6913453
allergys : ['밀']

al_food_json : {'allergy':'밀','food':'라면'}

al_food_dict : {'allergy': '밀', 'food': '라면'}

user_food : 라면

site : https://www.10000recipe.com/recipe/6913453

response : {'name': '냉라면', 'ingredients': '라면 1봉지, 양파 1/4개, 얼음 적당량, 설탕 1큰술, 식초 1큰술, 참기름 1큰술', 'recipe': ['양파는 얇게 채 썰어 찬물에 담가 매운

In [34]:
user1.allergies,user1.user_food

(['게'], '피자')

In [30]:
print(cross_message)
print(warning_message)

{'I00668': [], 'I00319': [], 'I00005': [], 'I00908': [], 'I00302': [], 'I00006': [], 'I01993': [], 'I02290': []}
{'I00668': [], 'I00319': [], 'I00005': [], 'I00908': [], 'I00302': [], 'I00006': [], 'I01993': [], 'I02290': []}


In [25]:
print('{')
for a,b in cross_message.items():
    print(f"'{a}': {b},")
print('},')
print('{')
for a,b in warning_message.items():
    print(f"'{a}': {b},")
print('}')

{
},
{
}


In [26]:
print(warn_comment_li)
print()
print(cross_comment_li)

NameError: name 'warn_comment_li' is not defined

In [ ]:
cross_comment_li

In [ ]:
user1.allergies,user1.user_food

In [ ]:
al_food_json

In [ ]:
al_food_dict

In [31]:
print(f'allergys : {allergys}')
print()
print(f'al_food_json : {al_food_json}')
print()
print(f'al_food_dict : {al_food_dict}')
print()
print(f'user_food : {user_food}')
print()
print(f'recipe_num : {recipe_num}')
print()
print(f'url_num : {url_num}')
print()
print(f'site : {site}')
print()
print(f'response : {response}')
print()
# print(f'ingredients : {ingredients}')
# print()
# print(f'response4 : {response4}')
# print()
# print(f'ingredients_rename : {ingredients_rename}')
# print()
# # print(f'cross_foods : {cross_foods}')
# print()
# # print(f'result_cross_foods : {result_cross_foods}')

allergys : ['게']

al_food_json : {'allergy':'게','food':'피자'}

al_food_dict : {'allergy': '게', 'food': '피자'}

user_food : 피자

recipe_num : 20698

url_num : 6330304

site : https://www.10000recipe.com/recipe/6330304

response : {'name': '피자맛 샌드위치', 'ingredients': '식빵, 양파 1/2개, 파프리카 1/2개, 햄 50g, 옥수수캔 2숟갈, 피자치즈 약간, 올리브 오일 적당량, 케첩 3숟갈', 'recipe': ['양파와 빨간 파프리카, 햄을 잘게 다져 준다.', '팬에 오일을 두르고 재료를 볶다가 케첩을 넣고 볶는다.', '샌드위치 메이커에 식빵을 올리고 피자 치즈와 속재료를 올린 후 다시 식빵을 덮는다.', '샌드위치 메이커를 닫고 3~4분간 기다린 후 피자맛 샌드위치를 즐긴다.']}



In [32]:
context

[Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 1276}, page_content='레시피번호: 1410\n요리명: 피자\n요리제목: 만들기 쉽고 달콤한 크리스마스 피자\nurl일련번호: 944375'),
 Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 28995}, page_content='레시피번호: 30041\n요리명: 떠먹는피자\n요리제목: 토마토와 고구마  감자로 만든 간편요리 떠먹는 피자\nurl일련번호: 6833055'),
 Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 21732}, page_content='레시피번호: 22655\n요리명: 피자\n요리제목: 스타엣지피자 (별집피자) 만들기\nurl일련번호: 6639259'),
 Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 153810}, page_content='레시피번호: 155930\n요리명: 피자\n요리제목: 김 신영...... 빼고파다이어트 피자...\nurl일련번호: 6982947'),
 Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 140567}, page_content='레시피번호: 142646\n요리명: 또띠아피자\n요리제목: 류수영 토띠아 피자 토치로 불맛낸 15분피자\nurl일련번호: 6968452'),
 Document(metadata={'source': 'data/레시피이름_url번호.csv', 'row': 39089}, page_content='레시피번호: 40323\n요리명: 토마토바질페스토피자\n요리제목: 향긋한 바질과 상큼한 토마토가 어우러진 또띠아바질페스토 피자\nurl일련번호: 6846128'),
 Document(metadata={'source': 'data/

In [ ]:
print(f'allergys : {allergys}')
print()
print(f'al_food_json : {al_food_json}')
print()
print(f'al_food_dict : {al_food_dict}')
print()
print(f'user_food : {user_food}')
print()
print(f'site : {site}')
print()
print(f'response : {response}')
print()
# print(f'ingredients : {ingredients}')
# print()
# print(f'response4 : {response4}')
# print()
# print(f'ingredients_rename : {ingredients_rename}')
# print()
# # print(f'cross_foods : {cross_foods}')
# print()
# # print(f'result_cross_foods : {result_cross_foods}')

In [ ]:
','.join(['돼지고기'])

In [ ]:
context

In [ ]:
user_food = None

In [ ]:
import re
c_temp = [doc for doc in context  if re.search(fr'{user_food}', doc.page_content)]
c_temp